In [1]:
import requests
import pandas as pd
import time
import json
from ipywidgets import IntProgress
from IPython.display import display
import glob
import re
from random import randint
import ast

In [2]:
URL = "https://api.leboncoin.fr/finder/search"

In [3]:
headers={   "api_key": "ba0c2dad52b3ec",
"Content-Type": "text/plain;charset=UTF-8",
"DNT": "1",
"Origin": "https://www.leboncoin.fr",
"Referer":"https://www.leboncoin.fr/recherche/?category=2&text=duster&owner_type=private&page=2",
'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"
}

In [4]:
def get_body(text_search,offset=0):
    body = {"filters":{"category":{"id":"2"},"enums":{"ad_type":["offer"]},"keywords":{"text":text_search,"type":"subject"},"location":{},"ranges":{}},"limit":35,"limit_alu":3,"owner_type":"private","offset":offset}
    return body

In [5]:
def get_total_pages(text_search):
    body= get_body(text_search)
    print(body)
    result = (requests.post(URL,json=body,headers=headers)).json()
    total_results = result['total']
    print(total_results)
    total_pages = int(total_results/35)
    return total_pages

In [6]:
def get_search(text_search,offset=0):
    time.sleep(30+(randint(1, 45)))
    print(f'offset ={offset}')
    try:
        result = (requests.post(URL,json=get_body(text_search,offset),headers=headers)).json()
        df = pd.DataFrame(result['ads'])
        print(f'df found : {df.shape}')
        return df
    except Exception as e:
        print(offset)
        print(e)
        return None
       
    

In [7]:
def get_all_offers(text_search):
    df_all = pd.DataFrame()
    total_pages = get_total_pages(text_search)
    page=0
    while page<=total_pages:
        df = get_search(text_search,page*35)
        df_all = pd.concat([df_all,df],axis=0)
        page = page+1
        print(f'(page={page})')
        print(f'shape fo df_all:{df_all.shape}')
        df_all.to_csv(f'leboncoin_{text_search}_pages_to_offset_{int(page*35)}.csv')

    df_all.to_csv(f'leboncoin_{str(text_search)}.csv')

    

In [8]:
get_all_offers('transporter')

{'filters': {'category': {'id': '2'}, 'enums': {'ad_type': ['offer']}, 'keywords': {'text': 'transporter', 'type': 'subject'}, 'location': {}, 'ranges': {}}, 'limit': 35, 'limit_alu': 3, 'owner_type': 'private', 'offset': 0}
403
offset =0
df found : (35, 19)
(page=1)
shape fo df_all:(35, 19)
offset =35
df found : (35, 19)
(page=2)
shape fo df_all:(70, 19)
offset =70
df found : (35, 19)
(page=3)
shape fo df_all:(105, 19)
offset =105
df found : (35, 19)
(page=4)
shape fo df_all:(140, 19)
offset =140
df found : (35, 19)
(page=5)
shape fo df_all:(175, 19)
offset =175
df found : (35, 19)
(page=6)
shape fo df_all:(210, 19)
offset =210
df found : (35, 19)
(page=7)
shape fo df_all:(245, 19)
offset =245
df found : (35, 19)
(page=8)
shape fo df_all:(280, 19)
offset =280
df found : (35, 19)
(page=9)
shape fo df_all:(315, 19)
offset =315
df found : (35, 19)
(page=10)
shape fo df_all:(350, 19)
offset =350
df found : (35, 19)
(page=11)
shape fo df_all:(385, 19)
offset =385
df found : (19, 19)
(page=

df_1 = pd.read_csv('leboncoin_duster_pages_to_offset_1155.csv')
df_2 = pd.read_csv('leboncoin_duster_pages_to_offset_1820.csv')
df = pd.concat([df_1,df_2],axis=0)
df.reset_index(inplace=True)
df_loc = df['location'].apply(lambda x: pd.Series(ast.literal_eval(x)))
df_loc.to_csv('duster_locations.csv')